In [2]:
import pandas as pd
import numpy as np
import pickle
import time
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import zipfile
import io

In [3]:
import tensorflow as tf
import keras
from scikeras.wrappers import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import KFold,train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

In [4]:
train = pd.read_csv('final/final.csv')
train.head()

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,0,1,2,3,4,5,6,7,8,9
0,2,3,66,348,48862,1,12,0,1,9,...,2.168369,-0.061117,0.723259,-0.254974,0.258598,0.862231,0.864002,0.177810,-0.043409,-0.065274
1,2,3,66,318,52078,3,756,0,1,4,...,1.081223,-0.472179,-0.336746,0.197516,0.078124,0.123741,-0.185023,-0.006560,0.071988,-0.086640
2,30,4,195,548,56440,3,1048,0,1,9,...,0.354725,0.776146,-0.230059,0.319022,0.138679,0.122441,-0.190630,0.179111,0.252992,-0.125460
3,30,4,195,991,47725,3,1048,0,0,9,...,0.739997,-0.263293,-0.169228,0.014864,-0.019029,-0.051152,0.003836,0.314507,0.090583,-0.201417
4,2,3,66,462,41898,1,1482,0,1,1,...,0.653804,0.306180,0.006980,-0.212106,-0.020089,0.003922,-0.167449,-0.051248,-0.155864,-0.055948


In [5]:
train['hotel_cluster'].value_counts()

hotel_cluster
91    100388
48     69653
42     61271
59     54680
28     54301
       ...  
35      5811
92      5593
53      3977
27      2303
74      2219
Name: count, Length: 100, dtype: int64

* Since our previous ML Models have been performing poorly, we decide to move to deep learning in order to achieve better performance.
* To achieve this, we consider the top 10 common clusters from our cleaned dataset 

In [6]:
most_common_clusters = list(train.hotel_cluster.value_counts().head(10).index)
most_common_clusters

[91, 48, 42, 59, 28, 16, 18, 95, 50, 21]

In [7]:
train = train.loc[train['hotel_cluster'].isin(most_common_clusters)]

In [8]:
print("Length of new training dataset:", len(train))

Length of new training dataset: 568972


In [9]:

train['hotel_cluster'].replace([91, 48, 42, 59, 28, 16, 18, 95, 50, 21],  
                               ['ninety-one','forty-eight','forty-two','fifty-nine','twenty-eight', 'sixteen',
                                'eighteen','ninety-five','fifty','twenty-one'], 
                               inplace=True)
train['hotel_cluster'].value_counts()

hotel_cluster
ninety-one      100388
forty-eight      69653
forty-two        61271
fifty-nine       54680
twenty-eight     54301
sixteen          46784
eighteen         46767
ninety-five      46755
fifty            45915
twenty-one       42458
Name: count, dtype: int64

In [26]:
X = train.drop(['hotel_cluster'], axis=1)
y_target = train['hotel_cluster']

In [28]:
from sklearn.preprocessing import LabelEncoder

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_target)
encoded_y = encoder.transform(y_target)
# convert integers to dummy variables (i.e. one hot encoded)
y = keras.utils.to_categorical(encoded_y)


In [25]:
print(encoded_y)

[5 5 5 ... 6 4 4]


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [13]:
# scales data between 0 and 1
X_train = keras.utils.normalize(X_train, axis=1)  
X_test = keras.utils.normalize(X_test, axis=1)

In [14]:
X_train.shape


(455177, 38)

#### KERAS MODEL

In [15]:
def c_model():

  model = Sequential()
  model.add(Dense(2048, activation='relu', input_dim=38))
  # model.add(Dense(1024, activation='relu'))
  # model.add(Dropout(0.25))
  model.add(Dense(512, activation='relu'))
  # model.add(Dropout(0.25))
  model.add(Dense(256, activation='relu'))
  # model.add(Dropout(0.25))
  # model.add(Dense(128, activation='relu'))
  # model.add(Dropout(0.25))
  model.add(Dense(64, activation='relu'))
  # model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))
  
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [16]:
model = KerasClassifier(build_fn=c_model, epochs=100, batch_size=128)

In [17]:
model.fit(X_train, y_train)

c:\Users\Dell\Desktop\RBS\Algorithmic Machine Learning\Project\Code\venv\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)



Epoch 1/100


3557/3557 [==============================] - 51s 14ms/step - loss: 2.2626 - accuracy: 0.1768
Epoch 2/100
3557/3557 [==============================] - 59s 17ms/step - loss: 2.2306 - accuracy: 0.1842
Epoch 3/100
3557/3557 [==============================] - 57s 16ms/step - loss: 2.1959 - accuracy: 0.2009
Epoch 4/100
3557/3557 [==============================] - 56s 16ms/step - loss: 2.1807 - accuracy: 0.2069
Epoch 5/100
3557/3557 [==============================] - 57s 16ms/step - loss: 2.1733 - accuracy: 0.2101
Epoch 6/100
3557/3557 [==============================] - 61s 17ms/step - loss: 2.1682 - accuracy: 0.2122
Epoch 7/100
3557/3557 [==============================] - 52s 15ms/step - loss: 2.1621 - accuracy: 0.2136
Epoch 8/100
3557/3557 [==============================] - 49s 14ms/step - loss: 2.1573 - accuracy: 0.2152
Epoch 9/100
3557/3557 [==============================] - 54s 15ms/step - loss: 2.1524 - accuracy: 0.2168
Epoch 10/100
3557/3557 [============================

KerasClassifier(
	model=None
	build_fn=<function c_model at 0x000001FCE2BB4DC0>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=128
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=100
	class_weight=None
)

### Grid Search CV

Tune batch size and epochs

In [18]:
model = KerasClassifier(build_fn=c_model)

batch_sizes = [128] #batch_size = 128
epochs = [30] #, 50, 100, 200]
parameters = {'epochs': epochs}
clf = GridSearchCV(model, parameters, cv=2)
clf.fit(X_train, y_train)

c:\Users\Dell\Desktop\RBS\Algorithmic Machine Learning\Project\Code\venv\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
7113/7113 [==============================] - 90s 12ms/step - loss: 2.2661 - accuracy: 0.1757
Epoch 2/30
7113/7113 [==============================] - 105s 15ms/step - loss: 2.2557 - accuracy: 0.1759
Epoch 3/30
7113/7113 [==============================] - 89s 12ms/step - loss: 2.2402 - accuracy: 0.1769
Epoch 4/30
7113/7113 [==============================] - 76s 11ms/step - loss: 2.2290 - accuracy: 0.1840
Epoch 5/30
7113/7113 [==============================] - 74s 10ms/step - loss: 2.2093 - accuracy: 0.1943
Epoch 6/30
7113/7113 [==============================] - 79s 11ms/step - loss: 2.1964 - accuracy: 0.2001
Epoch 7/30
7113/7113 [==============================] - 96s 14ms/step - loss: 2.1878 - accuracy: 0.2032
Epoch 8/30
7113/7113 [==============================] - 99s 14ms/step - loss: 2.1849 - accuracy: 0.2046
Epoch 9/30
7113/7113 [==============================] - 99s 14ms/step - loss: 2.1796 - accuracy: 0.2070
Epoch 10/30
7113/7113 [==============================] - 93s 13

c:\Users\Dell\Desktop\RBS\Algorithmic Machine Learning\Project\Code\venv\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
7113/7113 [==============================] - 94s 13ms/step - loss: 2.2642 - accuracy: 0.1777
Epoch 2/30
7113/7113 [==============================] - 100s 14ms/step - loss: 2.2566 - accuracy: 0.1778
Epoch 3/30
7113/7113 [==============================] - 88s 12ms/step - loss: 2.2350 - accuracy: 0.1825
Epoch 4/30
7113/7113 [==============================] - 88s 12ms/step - loss: 2.2127 - accuracy: 0.1939
Epoch 5/30
7113/7113 [==============================] - 86s 12ms/step - loss: 2.1978 - accuracy: 0.2004
Epoch 6/30
7113/7113 [==============================] - 96s 14ms/step - loss: 2.1896 - accuracy: 0.2036
Epoch 7/30
7113/7113 [==============================] - 103s 15ms/step - loss: 2.1830 - accuracy: 0.2065
Epoch 8/30
7113/7113 [==============================] - 103s 14ms/step - loss: 2.1788 - accuracy: 0.2082
Epoch 9/30
7113/7113 [==============================] - 100s 14ms/step - loss: 2.1742 - accuracy: 0.2102
Epoch 10/30
7113/7113 [==============================] - 98s

c:\Users\Dell\Desktop\RBS\Algorithmic Machine Learning\Project\Code\venv\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
14225/14225 [==============================] - 183s 13ms/step - loss: 2.2564 - accuracy: 0.1768
Epoch 2/30
14225/14225 [==============================] - 199s 14ms/step - loss: 2.2354 - accuracy: 0.1792
Epoch 3/30
14225/14225 [==============================] - 191s 13ms/step - loss: 2.2071 - accuracy: 0.1955
Epoch 4/30
14225/14225 [==============================] - 150s 11ms/step - loss: 2.1888 - accuracy: 0.2028
Epoch 5/30
14225/14225 [==============================] - 129s 9ms/step - loss: 2.1783 - accuracy: 0.2079
Epoch 6/30
14225/14225 [==============================] - 156s 11ms/step - loss: 2.1699 - accuracy: 0.2115
Epoch 7/30
14225/14225 [==============================] - 159s 11ms/step - loss: 2.1623 - accuracy: 0.2139
Epoch 8/30
14225/14225 [==============================] - 226s 16ms/step - loss: 2.1580 - accuracy: 0.2152
Epoch 9/30
14225/14225 [==============================] - 208s 15ms/step - loss: 2.1537 - accuracy: 0.2162
Epoch 10/30
14225/14225 [=============

GridSearchCV(cv=2,
             estimator=KerasClassifier(build_fn=<function c_model at 0x000001FCE2BB4DC0>),
             param_grid={'epochs': [30]})

In [19]:

print(clf.best_score_, clf.best_params_)
means = clf.cv_results_['mean_test_score']
parameters = clf.cv_results_['params']
for mean, parameter in zip(means, parameters):
  print(mean, parameter)

0.21650477971486337 {'epochs': 30}
0.21650477971486337 {'epochs': 30}


### SVM

In [29]:
X_svm_train, X_svm_test, y_svm_train, y_svm_test = train_test_split(X, y_target, test_size=0.2, random_state=1)

In [30]:
from sklearn import svm

clf = svm.SVC(kernel='rbf')
#Train the model using the training sets
clf.fit(X_svm_train, y_svm_train)

#Predict the response for test dataset
y_pred = clf.predict(X_svm_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy:", accuracy_score(y_svm_test, y_pred))
print("Precision:",precision_score(y_svm_test, y_pred, average="micro"))
print("Recall:",recall_score(y_svm_test, y_pred,  average="micro"))